In [16]:
import cv2
import mediapipe as mp
from tqdm import tqdm
import time

In [17]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose(static_image_mode=False,
                    model_complexity=2,
                    smooth_landmarks=True,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

In [18]:
def process_frame(img):
    # 记录该顿开始处理的时间
    start_time = time.time()
    #获取图像宽高
    h, w = img.shape[0],img.shape[1]
    # BGR转RGB
    img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)# 将RGB图像输入模型获取预测结果
    results = pose.process(img_RGB)
    if results.pose_landmarks:# 若检测出人体关键点来可视化关键点及骨架连线
        mp_drawing.draw_landmarks(img, results.pose_landmarks,mp_pose.POSE_CONNECTIONS)
        for i in range(33): # 遍历所有33个关键点，可视化
            #获取该关键点的三维坐标
            cx = int(results.pose_landmarks.landmark[i].x * w)
            cy = int(results.pose_landmarks.landmark[i].y * h)
            cz = results.pose_landmarks.landmark[i].z
            radius = 10
            if i== 0:#鼻尖
                img = cv2.circle(img,(cx,cy), radius,(0,0,255),-1)
            elif i in [11,12]:
                img = cv2.circle(img,(cx,cy), radius,(223,155,6),-1)
            elif i in [23,24]:
                img = cv2.circle(img,(cx,cy), radius,(1,240,255),-1)
            elif i in [13,14]:#胳膊时
                img = cv2.circle(img,(cx,cy), radius,(140,47,240),-1)
            elif i in [25,26]:
                img = cv2.circle(img,(cx,cy),radius,(0,0,255),-1)
            elif i in [15,16,27,28]:
                img = cv2.circle(img,(cx,cy), radius,(223,155,60),-1)
            elif i in [17,19,21]:
                img = cv2.circle(img,(cx,cy), radius,(94,218,121),-1)
            elif i in [18,20,22]:
                img = cv2.circle(img,(cx,cy),radius,(16,144,247),-1)
            elif i in [27,29,31]:
                img = cv2.circle(img,(cx,cy), radius,(29,123,243),-1)
            elif i in [28,30,32]:
                imq = cv2.circle(img,(cx,cy), radius, (193,182,255), -1)
            elif i in [9,10]:#嘴
                img = cv2.circle(img,(cx,cy), radius,(205,235,255),-1)
            elif i in [1,2,3,4,5,6,7,8]:
                img = cv2.circle(img,(cx,cy), radius, (94,218,121),-1)
            else:
                img = cv2.circle(img,(cx,cy), radius, (0,255,0),-1)
    else:
        scaler = 1
        failure_str = 'No Person'
        img = cv2.putText(img, failure_str,(25 * scaler,100 * scaler),cv2.FONT_HERSHEY_SIMPLEX,1.25*scaler,(255,255,255),1)

    end_time = time.time()
    FPS = 1/(end_time - start_time)
    scaler = 1#在图像上写FPS数值，参数依次为: 图片，添加的文字，左上角坐标，字体，字体大小，颜色，字体粗细
    img = cv2.putText(img,'FPS'+str(int(FPS)),(25 * scaler,50 * scaler),cv2.FONT_HERSHEY_SIMPLEX,1.25 * scaler,(255,255,255),1)
    return img

In [19]:
def generate_video(input_path='../datasets/example/jntm.mp4'):
    filehead = input_path.split('/')[-1]
    output_path ='../datasets/example/out-' + filehead
    print('视频开始处理',input_path)
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    # cv2.namedwindow('Crack Detection and Measurement Video Processing'
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH),
                  cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # fourcc= int(cap.get(cV2.CAP PROP FOURCC))# fourcc= cv2.VideoWriter fourcc(*'XVID')
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success,frame = cap.read()
                if not success:
                    break
                try:
                    frame = process_frame(frame)
                except :
                    print('error')
                    pass

                if success == True:
                    out.write(frame)
                    pbar.update(1)
        except:
            print('中途中断')
            pass
    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存',output_path)

In [20]:
generate_video(input_path='../datasets/example/jntm.mp4')

视频开始处理 ../datasets/example/jntm.mp4
视频总帧数为 1479


100%|██████████| 1478/1478 [04:46<00:00,  5.16it/s]

视频已保存 ../datasets/example/out-jntm.mp4
